In [2]:
import numpy as np
from matplotlib import pylab as plt
%autosave 0
%matplotlib inline

Autosave disabled


In [3]:
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [4]:
import pandas as pd
import gzip
import json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('./Video_Games_5.json.gz')



In [5]:
X = df['reviewText']
y = df['overall']

print(len(X.tolist()))

497577


Vectorize all of the reviews using bag of words
- Keras vectorization implementation: https://keras.io/api/layers/preprocessing_layers/core_preprocessing_layers/text_vectorization/
- Overview of algorithm by Google Cloud Platform: https://www.youtube.com/watch?v=UFtXy0KRxVI

In [20]:

# creates an object for each item inside of a given tensor, then adds them all into a list
text_dataset = tf.data.Dataset.from_tensor_slices(X.tolist()[:5000])

# max_features = 5000  # Maximum vocab size.
max_len = 4  # Sequence length to pad the outputs to.
embedding_dims = 2

# # Create the layer.  
vectorize_layer = TextVectorization(
max_tokens=max_features,
output_mode='binary')


# # Now that the vocab layer has been created, call `adapt` on the text-only  
# # dataset to create the vocabulary. You don't have to batch, but for large  
# # datasets this means we're not keeping spare copies of the dataset.  
vectorize_layer.adapt(text_dataset.batch(64))
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)

Run vectorized features through kNN classifier and measure accuracy
- features = vectorized full-reviews
- y labels = 'overall' ratings

In [28]:
input_data = [["this game is awful"], ["this game great"]]
foo = model.predict(input_data) # tells you if the words from input_data are part of the vocabulary (1 or 0)

print(foo)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
